# Template Matching 2

Ovaj dio koda kao i prethodni koristi template matching ali se vrtio na serveru. Na njemu smo radili tako da smo pomoću već označenih i cropanih logoa, kreirali dodatne skupove, a to je Croped logos za sve slike, te Validation u koji smo spremali slike za koje naš postupak nije bio dovoljno dobar da bi otkrio ground truth logoa na slici. Taj skup smo kasnije imali namjeru koristiti kao validacijski skup za treniranje modela. (slike za koje znamo koje su kategorije, ali ne znamo gdje je točno logo)


In [ ]:
import cv2
import sys
import os
import numpy as np
#from skimage.measure import compare_ssim as ssim
import datetime
import json

NEWW = 220
NEWH = 400

#THRESH = (400*216)/75
EDITEDLOGOS = '/home/bosnmar/MachineLearning/Production/FinalLogos/'
SOURCEIMAGES = '/home/bosnmar/MachineLearning/mozgalo/'
CROPED = '/home/bosnmar/MachineLearning/Production/Croped/'

MARKEDLOGOS = '/home/bosnmar/MachineLearning/Production/MarkedLogos/'
VALIDATION = '/home/bosnmar/MachineLearning/Production/Validation/'
METHOD = 'cv2.TM_CCOEFF_NORMED'


def processPicture(name, ss, locations):
    template = cv2.imread(CROPED + ss,0)

    w, h = template.shape[::-1]
    method =eval(METHOD)

    image = cv2.imread(SOURCEIMAGES + ss + "/" + name)
    imOut = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    imOut = imOut[0:round((imOut.shape[1]/2)), 20:imOut.shape[0]-20]
    coeff = float(NEWH/image.shape[0])

    try:
        res = cv2.matchTemplate(imOut,template,method)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

        top_left = max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        image = cv2.resize(image, (NEWW, NEWH))

        x = round(coeff * top_left[0])
        y = round(coeff * top_left[1])
        w = round(coeff * (bottom_right[0] - top_left[0]))
        h = round(coeff * (bottom_right[1] - top_left[1]))
        locations[ss].append([ss+"_"+name, x,y,w,h])
        #print (x, y, w, h)
        #cv2.rectangle(image,(x,y), (x+w, y+h), 255, 2)
        #cv2.rectangle(imOut, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)

        #crop_img = image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]

        #crop_img = cv2.resize(crop_img, (32, 16))

        

        location = MARKEDLOGOS + ss + '/' + str(name)

        cv2.imwrite(location, image)
    except:
        print(name)
    

if __name__ == '__main__':

    SOURCE = [retail for retail in os.listdir(SOURCEIMAGES)]
    LOGI = [log for log in os.listdir(EDITEDLOGOS)]
    i = 0
    locations = {}
    start = datetime.datetime.now()
    for ss in SOURCE:
    
        print("Working on: {}".format(ss))
        print("Started at: {}".format(datetime.datetime.now()))
        locations.update({ss:[]})
        sourimg = [img for img in os.listdir(SOURCEIMAGES + ss + "/")]
        logimg = [log for log in os.listdir(EDITEDLOGOS + ss + "/")]

        if not os.path.exists(MARKEDLOGOS + ss):
            os.makedirs(MARKEDLOGOS+ss)
        
        if not os.path.exists(VALIDATION + ss):
            os.makedirs(VALIDATION + ss)
        
        for img in sourimg:
            i = i + 1
            if img not in logimg:
                my_img = cv2.imread(SOURCEIMAGES + ss + "/" + img)
                #my_img = cv2.resize(my_img, (NEWW, NEWH))
                cv2.imwrite(VALIDATION + ss + "/" + img, my_img)

            else:
                processPicture(img, ss, locations)
        
        print("Finished with: {}".format(ss))
        print("Finished at: {}\n".format(datetime.datetime.now()))
    
    end = datetime.datetime.now()

    print ("Started at: {}".format(start))
    print ("Ended at: {}".format(end)) 
    for key in locations.keys():
        print ("{}: {}".format(key, str(len(locations[key]))))

    with open('logo_coord.json', 'w') as outfile:
        json.dump(locations, outfile)    